# Crawl movies on Yahoo movies websites

In [20]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
import re

In [23]:
proxies = {'http': '160.16.221.135:3128', 'https': '160.16.221.135:3128'}

In [21]:
# initialize a url list for movies 
urls = []

# access 49 pages for scrapying htmls of movies  
## ip will be blocked  sometime
for i in range(1,50):
    r = requests.get("https://movies.yahoo.co.jp/movie/?page="+str(i), proxies=proxies)
    soup = BeautifulSoup(r.content, "html.parser")
    tags = soup.find_all("li",{"class":"col"})
    for tag in tags:
        url="https://movies.yahoo.co.jp/movie/"+tag.get("data-cinema-id")+"/"
        urls.append(url)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [116]:
len(urls)

784

In [109]:
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")


In [115]:
image_words = []
image_list = soup.find("ul", class_="image_words list-inline text-xsmall").find_all("li")
for i in image_list:
    image_words.append(i.contents[0])
type(image_words)

list

In [117]:
movies = pd.DataFrame([], columns=["title", "year", "country", "director", "image_word", "synopsis", "score"])
cnt=0
for url in urls:
    cnt+=1
    print(cnt)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    
    title = re.sub(r" -(.)*","",soup.title.contents[0])
    
    try:
        director = soup.find("dl").find("span").contents[0]
    except AttributeError:
        director = ""
        print("non-drctry:", url)
    
    try:
        image_words = []
        image_list = soup.find("ul", class_="image_words list-inline text-xsmall").find_all("li")
        for i in image_list:
            image_words.append(i.contents[0])
    except AttributeError:
        print("non-imgwrd:", url)
        
    try:
        score = soup.find("span", class_="rating-score").find("span").contents[0]
    except AttributeError:
        score = ""
        print("non-scr:", url)
        
    try:
        year = soup.find(text="製作年度").find_next("td").contents[0][:4]
    except AttributeError:
        year = ""
        print("non-yr:", url)
        
    try:
        country = soup.find("ul", class_="list-inline no-space-bottom").find("li").contents[0]
    except AttributeError:
        country = ""
        print("non-cntry:", url)
    
    u = url + "story/"
    r = requests.get(u)
    soup = BeautifulSoup(r.content, "html.parser")
    try:
        synopsis = soup.find("p", class_="text-readable").contents[0][1:]
    except AttributeError:
        synopsis = ""
        print("non-synpss:", url)
        
    addRow = pd.DataFrame([title, year, country, director, image_words, synopsis, score], index=movies.columns).T
    movies = movies.append(addRow)
    movies_copy = movies.copy()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
non-drctry: https://movies.yahoo.co.jp/movie/368604/
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
non-drctry: https://movies.yahoo.co.jp/movie/365582/
148
149
150
non-drctry: https://movies.yahoo.co.jp/movie/363753/
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
non-scr: https://movies.yahoo.co.jp/movie/369746/
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225

TooManyRedirects: Exceeded 30 redirects.

In [198]:
url

'https://movies.yahoo.co.jp/movie/14317/'

In [118]:
movies = movies.sort_values("score", ascending=False)

,title,year,country,director,image_word,synopsis,score


In [119]:
movies = movies.drop(movies[movies.score == "未評価"].index, axis=0)
movies = movies.drop(movies[movies.score == ""].index, axis=0)
movies = movies.drop(movies[movies.synopsis == ""].index, axis=0)

In [104]:
movies = movies.drop(movies[movies.image_word.apply(lambda x: x==[])].index, axis=0)

In [122]:
len(movies)

0

In [106]:
movies = movies.drop_duplicates(subset={"title"}, keep="first")

In [107]:
movies = movies.reset_index(drop=True)
movies

,title,year,country,director,image_word,synopsis,score
0,夕陽のあと,2019,日本,越川道夫,"[[切ない], [泣ける]]",鹿児島県の長島町を舞台にしたヒューマンドラマ。1年前に島にやってきた女性と里子を育てる女性の...,5.00
1,ドクター・スリープ,2019,アメリカ,マイク・フラナガン,"[[泣ける], [不思議], [切ない]]",スティーヴン・キングのホラー小説をスタンリー・キューブリック監督がジャック・ニコルソン主演で...,5.00
2,ヴァイオレット・エヴァーガーデン 外伝,2019,日本,藤田春香,"[[泣ける], [切ない], [ロマンチック]]",2018年に放送され人気を博したアニメ「ヴァイオレット・エヴァーガーデン」の外伝。名門女学園...,4.58
3,映画 スター☆トゥインクルプリキュア 星のうたに想いをこめて,2019,日本,,"[[泣ける], [かわいい], [楽しい]]",アニメ『プリキュア』シリーズで、星座をモチーフにした「スター☆トゥインクルプリキュア」の劇場...,4.58
4,バック・トゥ・ザ・フューチャー,1985,アメリカ,ロバート・ゼメキス,"[[楽しい], [笑える], [かっこいい]]",1980年代を代表する大ヒット作であるSFアドベンチャーの傑作。自動車型タイムマシンで198...,4.58
5,ターミネーター2,1991,アメリカ,ジェームズ・キャメロン,"[[かっこいい], [勇敢], [泣ける]]",前作で結ばれた、サラ・コナーと未来から来たカイル・リースとの間に出来た息子ジョンこそ、未来...,4.55
6,冴えない彼女の育てかた Fine,2019,日本,柴田彰久,"[[かわいい], [笑える], [楽しい]]",丸戸史明(原作)と深崎暮人(イラスト)のライトノベルシリーズを原作にしたアニメの劇場版。主人...,4.54
7,ボヘミアン・ラプソディ,2018,"イギリス,アメリカ",ブライアン・シンガー,"[[泣ける], [かっこいい], [切ない]]",「伝説のチャンピオン」「ウィ・ウィル・ロック・ユー」といった数々の名曲で知られるロックバンド...,4.53
8,ショーシャンクの空に,1994,アメリカ,フランク・ダラボン,"[[泣ける], [知的], [勇敢]]",妻とその愛人を射殺したかどでショーシャンク刑務所送りとなった銀行家アンディ。初めは戸惑って...,4.53
9,Thunderbolt Fantasy 西幽げん歌,2019,"日本,台湾",,"[[かっこいい], [ファンタジー], [勇敢]]",台湾で国民的な人気の人形劇を、虚淵玄が原案・脚本・総監修を担当して映像化した日本・台湾の合作...,4.52


In [108]:
movies.to_csv("./movies_data.csv", encoding="utf-8")